Multi-temporal cloud-free image composites from Sentinel-2 MSI L2A
  
  - Gather the Sentinel-2 collection for 2019 across the Southern Rockies
  - Identify occluded pixels using the Cloud Score + (REF)
  - Calculate spectral indices across the time-series
  - Calculate the median pixel values
  - Set attributes and export the collection

Author: Maxwell C. Cook
maxwell.cook@colorado.edu

In [ ]:
# Initialize the GEE API and other libraries
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='jfsp-aspen')

In [ ]:
# Function to create grid
def make_grid(geo, res):
    proj = ee.Projection('EPSG:32635')
    lonlat = ee.Image.pixelLonLat()
    lon_grid = lonlat.select('longitude').multiply(100000).toInt()
    lat_grid = lonlat.select('latitude').multiply(100000).toInt()
    return lon_grid.multiply(lat_grid).reduceToVectors(geometry=geo, scale=res, geometryType='polygon')

# Define Southern Rockies ecoregion and create grid

res = 50000

ecoregions = ee.FeatureCollection("EPA/Ecoregions/2013/L3")

srme = ecoregions.filter(
    ee.Filter.eq('na_l3name', 'Southern Rockies')
)
bounds = srme.geometry().bounds().buffer(res)

grid = make_grid(bounds, res).filterBounds(srme).map(
    lambda ftr: ftr.set('grid_id', ftr.get('system:index')))

# Initialize a map
Map = geemap.Map()
Map.addLayerControl()

# Style dictionaries
grid_style = {'color': 'blue', 'fillColor': '00000000'}
srme_style = {'color': 'green', 'fillColor': '00000000'}

# Adding layers with styles
Map.addLayer(grid.style(**grid_style), {}, 'Spatial Block Grid')
Map.addLayer(srme.style(**srme_style), {}, 'Southern Rockies')

Map.centerObject(srme, 6)
Map

Calculates Spectral Indices from Sentinel-2 MSI Level-2A Imagery

  - Chlorophyll Index Red-edge (CIRE) [1]
  - Specific Leaf Area Vegetation Index (SLAVI) [2]
  - Inverted Red-edge Chlorophyll Index (IRECI) [3]
  - Modified Chlorophyll Absorption in Reflectance Index (MCARI) [4]
  - Red-edge Normalized Difference Vegetation Index (NDVI705) [5]

Author: Maxwell C. Cook,

 PhD Student, Department of Geography,

 University of Colorado Boulder

 maxwell.cook@colorado.edu

Sources:

[1] Gitelson, A.A., Gritz †, Y., Merzlyak, M.N., 2003.
    Relationships between leaf chlorophyll content and spectral reflectance and algorithms
    for non-destructive chlorophyll assessment in higher plant leaves.
    Journal of Plant Physiology 160, 271–282.
    https://doi.org/10.1078/0176-1617-00887

[2] Kobayashi, N., Tani, H., Wang, X., Sonobe, R., 2020.
    Crop classification using spectral indices derived from Sentinel-2A imagery.
    Journal of Information and Telecommunication 4, 67–90.
    https://doi.org/10.1080/24751839.2019.1694765

[3] Frampton, W.J., Dash, J., Watmough, G., Milton, E.J., 2013.
    Evaluating the capabilities of Sentinel-2 for quantitative estimation of biophysical variables in vegetation.
    ISPRS Journal of Photogrammetry and Remote Sensing 82, 83–92.
    https://doi.org/10.1016/j.isprsjprs.2013.04.007

[4] Dobrinić, D., Gašparović, M., Medak, D., 2021.
    Sentinel-1 and 2 Time-Series for Vegetation Mapping Using Random Forest Classification: A Case Study of Northern Croatia.
    Remote Sensing 13, 2321.
    https://doi.org/10.3390/rs13122321

[5] Evangelides, C., Nobajas, A., 2020.
    Red-Edge Normalised Difference Vegetation Index (NDVI705) from Sentinel-2 imagery to assess post-fire regeneration.
    Remote Sensing Applications: Society and Environment 17, 100283.
    https://doi.org/10.1016/j.rsase.2019.100283

[6] Xu, H., 2006.
    Modification of normalised difference water index (NDWI) to enhance open water features in remotely sensed imagery.
    International Journal of Remote Sensing 27, 3025–3033.
    https://doi.org/10.1080/01431160600589179

In [ ]:
def scale_nd(image):
    scale = ee.Number(32767.5)
    offset = ee.Number(1)
    return image.add(offset).multiply(scale).toUint16()

def scale_cire_slavi(image):
    scale = ee.Number(1000)
    offset = ee.Number(1)
    return image.add(offset).multiply(scale).toUint16()

def scale_ireci(image):
    offset = ee.Number(10000)
    scale = ee.Number(0.1)
    return image.add(offset).multiply(scale).toUint16()

def scale_mcari(image):
    offset = ee.Number(10000)
    scale = ee.Number(0.01)
    return image.add(offset).multiply(scale).toUint16()

def add_indices(image):
    # Band variables
    b2 = image.select('B2')  # Blue
    b3 = image.select('B3')  # Green
    b4 = image.select('B4')  # Red
    b5 = image.select('B5')  # Red-edge 1
    b6 = image.select('B6')  # Red-edge 2
    b7 = image.select('B7')  # Red-edge 3
    b8 = image.select('B8')  # NIR
    b8a = image.select('B8A')  # Red-edge 4
    b11 = image.select('B11')  # SWIR 1
    b12 = image.select('B12')  # SWIR 2

    # Spectral indices calculations
    cire = image.expression(
        '(N / RE1) - 1',
         {'N': b8, 'RE1': b5}).rename('CIRE')
    cire = scale_cire_slavi(cire)

    slavi = image.expression(
        'N / (Red + SWIR2)',
         {'N': b8, 'Red': b4, 'SWIR2': b12}).rename('SLAVI')
    slavi = scale_cire_slavi(slavi)

    ireci = image.expression(
        '(N - Red) / (RE1 / RE2)',
         {'N': b8, 'Red': b4, 'RE1': b5, 'RE2': b6}).rename('IRECI')
    ireci = scale_ireci(ireci)

    mcari = image.expression(
        '((RE1 - Red) - 0.2 * (RE1 - Green)) * (RE1 / Red)',
         {'RE1': b5, 'Red': b4, 'Green': b3}).rename('MCARI')
    mcari = scale_mcari(mcari)

    ndvi705 = image.normalizedDifference(['B6', 'B5']).rename('NDVI705')
    ndvi705 = scale_nd(ndvi705)

    mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
    mndwi = scale_nd(mndwi)

    # Return the image with added bands
    return image.addBands([slavi, cire, ireci, mcari, ndvi705, mndwi])

In [ ]:
# Load the data
s2l2a = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
csPlus = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

# Sentinel-2 MSI Bands and Vegetation Indices
s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
vi_bands = ['SLAVI', 'CIRE', 'IRECI', 'MCARI', 'NDVI705', 'MNDWI']

# Function to concatenate strings (rename bands)
def string_cat(item, y):
    return ee.String(item).cat(y)

def apply_string_cat(x, y):
    return x.map(lambda item: string_cat(item, y))

# Function to join collections
def join_collections(col1, col2):
    joined = ee.ImageCollection(ee.Join.saveFirst('cs').apply(
        primary=col1,
        secondary=col2,
        condition=ee.Filter.equals(
            leftField='system:index',
            rightField='system:index'
        )
    ))
    return joined.map(lambda image: image.addBands(image.get('cs')))


# Define some constants
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.60
YEAR = 2019

def generate_composites(grid_id):
  # Get the grid and geometry
  Grid = grid.filter(ee.Filter.eq('grid_id',grid_id))
  roi = Grid.geometry().buffer(1000)

  ###################
  # Summer Composite (Mid Greenup Phase to Onset Greenness Decrease)

  # Filters
  summer_filter = ee.Filter.And(
      ee.Filter.bounds(roi),
      ee.Filter.calendarRange(YEAR, YEAR, 'year'),
      ee.Filter.calendarRange(169, 242, 'DAY_OF_YEAR'),
      ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80),
      ee.Filter.lt('SNOW_ICE_PERCENTAGE', 10)
  )

  # Image Collection
  s2_summer = s2l2a.filter(summer_filter).select(s2_bands)

  # Grab some statistics from the collection
  n_summer = s2_summer.size();
  start_date = ee.Date(s2_summer.first().get('system:time_start'));
  end_date = ee.Date(
      s2_summer.limit(1,'system:time_start',False)
        .first().get('system:time_start'));

  # Join the Cloud Score Plus
  csPlus_s = csPlus.filter(
    ee.Filter.And(
      ee.Filter.bounds(roi),
      ee.Filter.calendarRange(YEAR, YEAR, 'year'),
      ee.Filter.calendarRange(169, 242, 'DAY_OF_YEAR'),
    )
  ).select([QA_BAND])

  # Mask out occluded pixels

  s2_summer = join_collections(s2_summer,csPlus_s)
  s2_summer = s2_summer.map(
      lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD))
  ).select(s2_bands)

  # Calculate the spectral indices
  s2_summer = s2_summer.map(add_indices).median().clip(roi).toUint16()
  s2_summer = s2_summer.set(
      'grid_id',grid_id,
      'season','summer',
      'n_summer',n_summer,
      'year',YEAR,
      'start_date',start_date,
      'end_date',end_date
  )

  # Update the band names
  new_bands = apply_string_cat(s2_summer.bandNames(),"_summer");
  s2_summer = s2_summer.select(s2_summer.bandNames(),new_bands);

  ##################
  # Autumn Composite (Mid Senescence Phase to Onset of Greenness Minimum)

  # Filters
  autumn_filter = ee.Filter.And(
      ee.Filter.bounds(roi),
      ee.Filter.calendarRange(YEAR, YEAR, 'year'),
      ee.Filter.calendarRange(275, 307, 'DAY_OF_YEAR'),
      ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80),
      ee.Filter.lt('SNOW_ICE_PERCENTAGE', 5)
  )

  # Image Collection
  s2_autumn = s2l2a.filter(autumn_filter).select(s2_bands)

  # Grab some statistics from the collection
  n_autumn = s2_autumn.size();
  start_date = ee.Date(s2_autumn.first().get('system:time_start'));
  end_date = ee.Date(
      s2_autumn.limit(1,'system:time_start',False)
        .first().get('system:time_start'));

  # Join the Cloud Score Plus
  csPlus_s = csPlus.filter(
    ee.Filter.And(
      ee.Filter.bounds(roi),
      ee.Filter.calendarRange(YEAR, YEAR, 'year'),
      ee.Filter.calendarRange(275, 307, 'DAY_OF_YEAR'),
    )
  ).select([QA_BAND])

  # Mask out occluded pixels
  s2_autumn = join_collections(s2_autumn,csPlus_s)
  s2_autumn = s2_autumn.map(
      lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD))
  ).select(s2_bands)

  # Calculate the spectral indices
  s2_autumn = s2_autumn.map(add_indices).median().clip(roi).toUint16()
  s2_autumn = s2_autumn.set(
      'grid_id',grid_id,
      'season','autumn',
      'n_autumn',n_autumn,
      'year',YEAR,
      'start_date',start_date,
      'end_date',end_date
  )

  # Update the band names
  new_bands = apply_string_cat(s2_autumn.bandNames(),"_autumn");
  s2_autumn = s2_autumn.select(s2_autumn.bandNames(),new_bands);

  ##########
  # Combine into a single stack

  stack = s2_summer.addBands(s2_autumn)

  return stack

# Apply the function to generate composites
grid_list = grid.aggregate_array('grid_id')
# grid_list = ee.List(['-239+86','-240+86','-240+87','-239+87']);
img_cols = grid_list.map(lambda x: generate_composites(x))
# print(img_cols.limit.size().getInfo())

result = ee.ImageCollection.fromImages(img_cols)

print("Done!")

Done!


In [ ]:

# # Define visualization parameters
# vis_params_summer = {
#     'bands': ['B4_summer', 'B3_summer', 'B2_summer'],
#     'min': 0,
#     'max': 3000,
#     'gamma': 1.4
# }

# vis_params_autumn = {
#     'bands': ['B4_autumn', 'B3_autumn', 'B2_autumn'],
#     'min': 0,
#     'max': 3000,
#     'gamma': 1.4
# }

# Map.addLayer(result.mean(), vis_params_summer, 'Seasonal Composite Summer')
# Map.addLayer(result.mean(), vis_params_autumn, 'Seasonal Composite Autumn')

# Map


KeyboardInterrupt: 

In [ ]:
def export_batch(collection, asset_folder, suffix, n):
    for i in range(n):
        img = ee.Image(collection.toList(n).get(i))
        grid_id = grid_list.get(i)
        grid_name = ee.String(grid_id).replace('-', '').replace('\\+', '').getInfo()
        grid_feature = grid.filter(ee.Filter.eq('grid_id',grid_id))
        region = grid_feature.geometry().buffer(1000)
        asset_id = f'{asset_folder}S2MSI_L2A_{suffix}_{grid_name}'
        task = ee.batch.Export.image.toAsset(
            image=img,
            description=f'S2MSI_L2A_{suffix}_{grid_name}',
            assetId=asset_id,
            region=region,
            scale=10,
            crs='EPSG:32613',
            maxPixels=1e13)
        task.start()

# Run the export
nn = grid_list.size().getInfo()
print(f"Exporting a grand total of !!{nn}!! grids")
asset_folder = 'projects/cires-gg-earthlab/aspen-mapping/S2MSI/'
export_batch(result, asset_folder, 'y2019_', nn)

Exporting a grand total of !!129!! grids
